# Transfer Öğrenme: Hazır Modelleri Kullanmak
a. Transfer Öğrenme Nedir?
Transfer Öğrenme (Transfer Learning), önceden büyük veri setlerinde (örneğin ImageNet) eğitilmiş bir modeli, yeni bir görev için yeniden kullanma tekniğidir.

### Popüler Hazır Modeller

- **VGG16 / VGG19**: Basit, anlaşılır yapı. İyi özellik çıkarır.
- **ResNet50**: Derin ağlar için atık bağlantılar (skip connections) kullanır.
- **MobileNet**: Mobil cihazlar için optimize edilmiştir, hafiftir.
- **EfficientNet**: Yüksek doğruluk, düşük hesaplama maliyeti.

In [9]:
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
base_dir = './cats_and_dogs_filtered'
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_dir,
                     batch_size=16,
                     class_mode='binary',
                     target_size=(224,224))

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                     batch_size=16,
                     class_mode='binary',
                     target_size=(224,224))

base_model = VGG16(weights='imagenet',
include_top=False,input_shape=(224,224,3))


base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1,activation='sigmoid')
])


model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples//validation_generator.batch_size)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/5


/Users/ibrahimediz/anaconda3/envs/ortamCV/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 268s 2s/step - accuracy: 0.7890 - loss: 0.8558 - val_accuracy: 0.9647 - val_loss: 0.1016
Epoch 2/5
  1/125 ━━━━━━━━━━━━━━━━━━━━ 3:26 2s/step - accuracy: 0.9375 - loss: 0.3611

In [ ]:
test_loss,test_acc = model.evaluate(validation_generator)
print('Test accuracy: ',test_acc)
print('Test loss: ',test_loss)


## Fine Tuning


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(1e-5),
loss="binary_crossentropy",metrics=["accuracy"])

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples//validation_generator.batch_size)